In [103]:
import pandas as aju
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
warnings.filterwarnings('ignore')

In [15]:
df = aju.read_csv('D:\MACHINE LEARNING\DATASETS\ML_18_Bagging_Heart.csv')

In [17]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [19]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [37]:
df[df.Cholesterol> (df.Cholesterol.mean()+3*df.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [41]:
df1 = df[df.Cholesterol <= (df.Cholesterol.mean()+3*df.Cholesterol.std())]
df1.shape

(915, 12)

In [43]:
df1[df1.MaxHR > (df1.MaxHR.mean()+3*df1.MaxHR.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [45]:
df1[df1.Oldpeak > (df1.Oldpeak.mean()+3*df1.Oldpeak.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.2,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.2,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.4,Down,1


In [49]:
df2 = df1[df1.Oldpeak <= (df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df2.shape

(909, 12)

In [51]:
df2[df2.RestingBP > (df2.RestingBP.mean()+3*df2.RestingBP.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
109,39,M,ATA,190,241,0,Normal,106,N,0.0,Up,0
241,54,M,ASY,200,198,0,Normal,142,Y,2.0,Flat,1
365,64,F,ASY,200,0,0,Normal,140,Y,1.0,Flat,1
399,61,M,NAP,200,0,1,ST,70,N,0.0,Flat,1
592,61,M,ASY,190,287,1,LVH,150,Y,2.0,Down,1
732,56,F,ASY,200,288,1,LVH,133,Y,4.0,Down,1
759,54,M,ATA,192,283,0,LVH,195,N,0.0,Up,1


In [55]:
df3 = df2[df2.RestingBP <= (df2.RestingBP.mean()+3*df2.RestingBP.std())]
df3.shape

(902, 12)

In [57]:
df3.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [59]:
df3.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [61]:
df3.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [63]:
df3.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [71]:
dt = df3.copy()

In [73]:
dt.ExerciseAngina.replace({'N':0,'Y':1},inplace = True)

In [77]:
dt.ST_Slope.replace({'Up':0,'Flat':1,'Down':2},inplace=True)

In [79]:
dt.RestingECG.replace({'Normal':0,'ST':1,'LVH':2},inplace = True)

In [81]:
dt

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,0,172,0,0.0,0,0
1,49,F,NAP,160,180,0,0,156,0,1.0,1,1
2,37,M,ATA,130,283,0,1,98,0,0.0,0,0
3,48,F,ASY,138,214,0,0,108,1,1.5,1,1
4,54,M,NAP,150,195,0,0,122,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,0,132,0,1.2,1,1
914,68,M,ASY,144,193,1,0,141,0,3.4,1,1
915,57,M,ASY,130,131,0,0,115,1,1.2,1,1
916,57,F,ATA,130,236,0,2,174,0,0.0,1,1


In [87]:
df5 = aju.get_dummies(dt,drop_first = True).astype(int)
df5.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,0,172,0,0,0,0,1,1,0,0
1,49,160,180,0,0,156,0,1,1,1,0,0,1,0
2,37,130,283,0,1,98,0,0,0,0,1,1,0,0
3,48,138,214,0,0,108,1,1,1,1,0,0,0,0
4,54,150,195,0,0,122,0,0,0,0,1,0,1,0


In [89]:
df5

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,0,172,0,0,0,0,1,1,0,0
1,49,160,180,0,0,156,0,1,1,1,0,0,1,0
2,37,130,283,0,1,98,0,0,0,0,1,1,0,0
3,48,138,214,0,0,108,1,1,1,1,0,0,0,0
4,54,150,195,0,0,122,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,0,132,0,1,1,1,1,0,0,1
914,68,144,193,1,0,141,0,3,1,1,1,0,0,0
915,57,130,131,0,0,115,1,1,1,1,1,0,0,0
916,57,130,236,0,2,174,0,0,1,1,0,1,0,0


In [95]:
x = df5.drop('HeartDisease',axis = 'columns')
y = df5.HeartDisease

In [99]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale

array([[-1.42896269,  0.46089071,  0.85238015, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-0.47545956,  1.5925728 , -0.16132855, ..., -0.4836591 ,
         1.86750159, -0.22914788],
       [-1.74679706, -0.10495034,  0.79657967, ...,  2.06757196,
        -0.53547478, -0.22914788],
       ...,
       [ 0.37209878, -0.10495034, -0.61703246, ..., -0.4836591 ,
        -0.53547478, -0.22914788],
       [ 0.37209878, -0.10495034,  0.35947592, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-1.64085227,  0.3477225 , -0.20782894, ..., -0.4836591 ,
         1.86750159, -0.22914788]])

In [101]:
x_train,x_test,y_train,y_test = train_test_split(x_scale,y,test_size = 0.20,random_state = 99)

In [105]:
svc_score = cross_val_score(SVC(),x,y,cv= 5)
svc_score.mean()

0.6906445672191528

In [107]:
bag = BaggingClassifier(estimator = SVC(),max_samples = 0.8,n_estimators=100,random_state = 99)
svc_bag = cross_val_score(bag,x,y,cv = 5)
svc_bag.mean()

0.6828729281767957

In [109]:
dtc_score = cross_val_score(DecisionTreeClassifier(),x,y,cv= 5)
dtc_score.mean()

0.7194290976058931

In [113]:
bag = BaggingClassifier(estimator = DecisionTreeClassifier(),max_samples = 0.8,n_estimators=100,random_state = 99,oob_score = True)
dtc_bag = cross_val_score(bag,x,y,cv = 5)
dtc_bag.mean()

0.7959238796807858

In [115]:
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
rfc.score(x_test,y_test)

0.8895027624309392